In [ ]:
from typing import List, Optional

from sqlalchemy import ForeignKey
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import (
    DeclarativeBase,
    Mapped,
    Session,
    mapped_column,
    relationship,
)
from typing_extensions import Self


class Base(DeclarativeBase):
    pass


class Node(Base):
    __tablename__ = "node"

    id: Mapped[str] = mapped_column(primary_key=True)
    label: Mapped[Optional[str]] = mapped_column(nullable=True)
    language: Mapped[Optional[str]]
    sense_label: Mapped[Optional[str]]
    term_id: Mapped[Optional[str]] = mapped_column(ForeignKey("node.id"))
    site: Mapped[Optional[str]]
    path: Mapped[Optional[str]]
    site_available: Mapped[Optional[bool]]

    term: Mapped[Optional["Node"]] = relationship()

    @classmethod
    def from_dict(cls, data: dict, session: Optional[Session] = None) -> Self:
        id_ = data["@id"]
        instance = session.get(cls, id_)
        if instance:
            return instance
        instance = cls(
            id=id_,
            label=data["label"],
            language=data.get("language"),
            sense_label=data.get("sense_label"),
            term_id=data.get("term_id"),
            site=data.get("site"),
            path=data.get("path"),
            site_available=data.get("site_available"),
        )
        try:
            session.add(instance)
            session.commit()
            return instance
        except IntegrityError as e:
            session.rollback()
            # If we get an integrity error, someone else beat us to it
            # Try one more time to get the instance
            instance = session.get(cls, id_)
            if instance:
                return instance
            # If we still don't have an instance, something else went wrong
            raise e from None

    def __repr__(self) -> str:
        return f"Node(id='{self.id}', label='{self.label}', language='{self.language}')"


class Relation(Base):
    __tablename__ = "relation"

    id: Mapped[str] = mapped_column(primary_key=True)
    label: Mapped[str]
    symmetric: Mapped[bool] = mapped_column(default=False)

    @classmethod
    def from_dict(cls, data: dict, session: Optional[Session] = None) -> Self:
        id_ = data["@id"]
        instance = session.get(cls, id_)
        if instance:
            return instance
        instance = cls(
            id=id_,
            label=data["label"],
            symmetric=data.get("symmetric", False),
        )
        try:
            session.add(instance)
            session.commit()
            return instance
        except IntegrityError as e:
            session.rollback()
            # If we get an integrity error, someone else beat us to it
            # Try one more time to get the instance
            instance = session.get(cls, id_)
            if instance:
                return instance
            # If we still don't have an instance, something else went wrong
            raise e from None

    def __repr__(self) -> str:
        return f"Relation(id='{self.id}', label='{self.label}', symmetric={self.symmetric})"


class Source(Base):
    __tablename__ = "source"

    id: Mapped[str] = mapped_column(primary_key=True)
    contributor: Mapped[Optional[str]]
    process: Mapped[Optional[str]]
    activity: Mapped[Optional[str]]
    edge_id: Mapped[str] = mapped_column(ForeignKey("edge.id"))

    @classmethod
    def from_dict(cls, data: dict, session: Optional[Session] = None) -> Self:
        id_ = data["@id"]
        instance = session.get(cls, id_)
        if instance:
            return instance
        instance = cls(
            id=id_,
            contributor=data.get("contributor"),
            process=data.get("process"),
            activity=data.get("activity"),
            edge_id=data["edge_id"],
        )
        try:
            session.add(instance)
            session.commit()
            return instance
        except IntegrityError as e:
            session.rollback()
            # If we get an integrity error, someone else beat us to it
            # Try one more time to get the instance
            instance = session.get(cls, id_)
            if instance:
                return instance
            # If we still don't have an instance, something else went wrong
            raise e from None

    def __repr__(self) -> str:
        return f"Source(id='{self.id}', contributor='{self.contributor}')"


class Edge(Base):
    __tablename__ = "edge"

    id: Mapped[str] = mapped_column(primary_key=True)
    rel_id: Mapped[str] = mapped_column(ForeignKey("relation.id"))
    start_id: Mapped[str] = mapped_column(ForeignKey("node.id"))
    end_id: Mapped[str] = mapped_column(ForeignKey("node.id"))
    license: Mapped[Optional[str]]
    weight: Mapped[float] = mapped_column(default=1.0)
    dataset: Mapped[Optional[str]]
    surface_text: Mapped[Optional[str]]

    rel: Mapped[Relation] = relationship()
    start: Mapped[Node] = relationship(foreign_keys=[start_id])
    end: Mapped[Node] = relationship(foreign_keys=[end_id])
    sources: Mapped[List[Source]] = relationship()

    @classmethod
    def from_dict(cls, data: dict, session: Optional[Session] = None) -> Self:
        id_ = data["@id"]
        instance = session.get(cls, id_)
        if instance:
            return instance
        rel = Relation.from_dict(data["rel"], session=session)
        start = Node.from_dict(data["start"], session=session)
        end = Node.from_dict(data["end"], session=session)
        sources = [
            Source.from_dict({"edge_id": id_, **source}, session=session)
            for source in data["sources"]
        ]
        instance = cls(
            id=id_,
            rel=rel,
            start=start,
            end=end,
            sources=sources,
            license=data["license"],
            weight=data["weight"],
            dataset=data["dataset"],
            surface_text=data.get("surfaceText"),
        )
        try:
            session.add(instance)
            session.commit()
            return instance
        except IntegrityError as e:
            session.rollback()
            # If we get an integrity error, someone else beat us to it
            # Try one more time to get the instance
            instance = session.get(cls, id_)
            if instance:
                return instance
            # If we still don't have an instance, something else went wrong
            raise e from None

    def __repr__(self) -> str:
        return f"Edge(id='{self.id}', start='{self.start_id}', rel='{self.rel_id}', end='{self.end_id}')"


In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session


def main():
    # Create SQLite database in memory
    engine = create_engine("sqlite:///graph.db", echo=True)
    
    # Create all tables
    Base.metadata.create_all(engine)
    
    # Create a session
    session = Session(engine)
    
    try:
        # Example data: Creating a small graph about animals
        # First node: Cat
        cat_data = {
            "@id": "concept/cat",
            "label": "cat",
            "language": "en",
            "sense_label": "domestic cat",
            "site": "wikipedia",
            "path": "/wiki/Cat",
            "site_available": True
        }
        
        # Second node: Mammal
        mammal_data = {
            "@id": "concept/mammal",
            "label": "mammal",
            "language": "en",
            "sense_label": "class of vertebrate animals",
            "site": "wikipedia",
            "path": "/wiki/Mammal",
            "site_available": True
        }
        
        # Relation: IsA
        is_a_relation_data = {
            "@id": "relation/is_a",
            "label": "IsA",
            "symmetric": False
        }
        
        # Create the edge connecting cat to mammal
        edge_data = {
            "@id": "edge/cat_is_mammal",
            "rel": is_a_relation_data,
            "start": cat_data,
            "end": mammal_data,
            "license": "CC-BY-SA",
            "weight": 1.0,
            "dataset": "animal_taxonomy",
            "sources": [
                {
                    "@id": "source/cat_mammal_wiki",
                    "contributor": "wikipedia",
                    "process": "manual_entry",
                    "activity": "taxonomy_classification"
                }
            ]
        }
        
        # Create all objects using from_dict
        edge = Edge.from_dict(edge_data, session=session)
        
        # Query and print the created objects
        print("\nCreated objects:")
        print("Nodes:")
        for node in session.query(Node).all():
            print(f"- {node}")
        
        print("\nRelations:")
        for relation in session.query(Relation).all():
            print(f"- {relation}")
            
        print("\nEdges:")
        for edge in session.query(Edge).all():
            print(f"- {edge}")
            print(f"  Start: {edge.start.label}")
            print(f"  Relation: {edge.rel.label}")
            print(f"  End: {edge.end.label}")
            
        print("\nSources:")
        for source in session.query(Source).all():
            print(f"- {source}")
            
    finally:
        session.close()
        
    # Example of querying specific relationships
    with Session(engine) as session:
        # Find all cats
        cat_node = session.query(Node).filter(Node.label == "cat").first()
        
        # Find all relationships where cat is the start node
        cat_relationships = session.query(Edge).filter(Edge.start_id == cat_node.id).all()
        
        print("\nRelationships involving cat:")
        for rel in cat_relationships:
            print(f"- {rel.start.label} {rel.rel.label} {rel.end.label}")
            print(f"  Source: {rel.sources[0].contributor}")
            print(f"  Dataset: {rel.dataset}")

if __name__ == "__main__":
    main()